In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from long_covid.styling import hide_and_move_axis

In [ ]:
vitals = pd.read_feather('../data/02_processed/vitals_processed.feather')
rhr = vitals[vitals.vitalid == 65]

In [ ]:
vacc_data = pd.read_feather('../data/01_raw/vaccinations.feather')
test_data = pd.read_feather('../data/01_raw/tests.feather')
metadata = pd.merge(vacc_data, test_data, on='user_id')

In [ ]:
data = pd.merge(metadata, rhr, left_on='user_id', right_on='userid')
data['dt'] = (data.date - data.first_dose).dt.days

random_dates = pd.date_range(start=df.first_dose.min(),end=df.first_dose.max())
random_dates = np.random.choice(random_dates, len(df.user_id.unique()))
random_dates = pd.DataFrame(data={'user_id': df.user_id.unique(), 'random_first_dose': random_dates})

data = pd.merge(data, random_dates, on='user_id')
data['dt_random'] = (data.date - data.random_first_dose).dt.days

df = data[data.test_result == 'negative']

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3), sharex=True, sharey=True)

before = df[(df.dt < 0) & (df.dt > -60)]
before = pd.DataFrame(before.groupby('user_id').value.mean())

after = df[(df.dt < 90) & (df.dt > 30)]
after = pd.DataFrame(after.groupby('user_id').value.mean())

diff = pd.merge(before, after, on='user_id')
diff['diff'] = diff.value_y - diff.value_x

bar, x = np.histogram(diff['diff'], bins=np.arange(-10.5, 10.5, 1), density=True)

ax1.bar((x[1:] + x[:-1]) / 2, bar)
ax1.axvline(diff['diff'].mean(), c='k', lw=1.5, ls='--')
print(diff['diff'].mean())

x1 = diff['diff']

before = df[(df.dt_random < 0) & (df.dt_random > -60)]
before = pd.DataFrame(before.groupby('user_id').value.mean())
after = df[(df.dt_random < 90) & (df.dt_random > 30)]
after = pd.DataFrame(after.groupby('user_id').value.mean())

diff = pd.merge(before, after, on='user_id')
diff['diff'] = diff.value_y - diff.value_x

bar, x = np.histogram(diff['diff'], bins=np.arange(-10.5, 10.5, 1), density=True)

ax2.bar((x[1:] + x[:-1]) / 2, bar)
ax2.axvline(diff['diff'].mean(), c='k', lw=1.5, ls='--')
print(diff['diff'].mean())

x2 = diff['diff']

x1 = np.sort(x1)
x2 = np.sort(x2)
hide_and_move_axis(ax1)
hide_and_move_axis(ax2)

ax1.set_ylabel('Relative frequency of observation')
ax1.set_xlabel('Difference in average resting heart rate [bpm]\nbetween the two months before and after\nreceiving first vaccination dose')
ax2.set_xlabel('Difference in average resting heart rate [bpm]\nbetween any two subsequent months')

In [ ]:
q1 = []
q2 = []

for i in np.linspace(1, 99):
    q1.append(np.percentile(x1, i))
    q2.append(np.percentile(x2, i))
    
np.corrcoef(q1, q2)